In [32]:
import random

def generateData(machines=4, jobs=7, seed=0):
    PROC = [ [ 0 for m in range(machines) ] for j in range(jobs) ]
    random.seed(seed)
    for j in range(jobs):
        for m in range(machines):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [33]:
import pulp

def IP(PROC):

    steps = [ len(job) for job in PROC ]
    assert(min(steps)==max(steps))
    N_MACHINES = len(PROC[0])
    N_JOBS = len(PROC)


    def val(x):
        return int(pulp.value(x))

    def proc(job, machine):
        return pulp.lpSum( [ PROC[j][machine] * JJ[job][j] for j in range(N_JOBS) ] ) 

    JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)), lowBound=0, upBound=1, cat='Integer')
    Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')

    prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)
    prob += Stop[N_JOBS-1][N_MACHINES-1]

    # JJ is a permutation of the jobs
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
        prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

    for m in range(N_MACHINES):
        for j in range(N_JOBS):
            prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

    for m in range(N_MACHINES):
        for j in range(N_JOBS):
            prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            prob += Start[j][m] + proc(j,m) == Stop[j][m]

    solvers = pulp.listSolvers(onlyAvailable=True) 
    solver = pulp.getSolver(solvers[0], msg=0)
    prob.solve(solver)

    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
                
    return acc, int(pulp.value(prob.objective))

In [34]:
import numpy as np

def totalTime(PROC, seq, log=False):
    
    def isPermutation(seq):
        for i in range(len(seq)):
            if i not in seq:
                return False
        return True
    
    steps = [ len(job) for job in PROC ]
    assert(len(PROC) == len(seq))
    assert(isPermutation(seq))
    assert(min(steps)==max(steps))
    n_machines = len(PROC[0])
    n_jobs = len(PROC)
    wait = np.zeros([n_jobs, n_machines], dtype=int)
    idle = np.zeros([n_jobs, n_machines], dtype=int)
    start = np.zeros([n_jobs, n_machines], dtype=int)
    stop  = np.zeros([n_jobs, n_machines], dtype=int)
    proc = np.zeros([n_jobs, n_machines], dtype=int)
    for job in range(n_jobs):
        proc[job] = PROC[seq[job]]
    for job in range(n_jobs):
        for machine in range(n_machines):
            start[job, machine] = max(stop[job-1, machine] if job>0 else 0, stop[job, machine-1] if machine>0 else 0)
            wait[job, machine] = start[job, machine] - (stop[job, machine-1] if machine>0 else 0)
            idle[job, machine] = start[job, machine] - (stop[job-1, machine] if job>0 else 0)
            stop[job, machine] = start[job, machine] + proc[job, machine]

    if log:

        row = '|---------|'
        for m in range(n_machines):
            row += f"------------|" \
                    f"------------|"
        print(row)
        
        row = '|         |'
        for m in range(n_machines):
            row += f"            |" \
                    f" Machine: {m:1d} |"
        print(row)
        
        for j in range(n_jobs):
            
            row = '|---------|'
            for m in range(n_machines):
                row += f"------------|" \
                       f"------------|"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Idle: {idle[j,m]:4d} |"
            print(row)
            
            row = '|---------|'
            for m in range(n_machines):
                row += f"------------|" \
                       f"------------|"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Start:{start[j,m]:4d} |"  
            print(row)
            
            row = f'| Job: {seq[j]:2d} |'
            for m in range(n_machines):
                row += f" Wait: {wait[j,m]:4d} | " \
                       f"Proc: {proc[j,m]:4d} |"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Stop: {stop[j,m]:4d} |"  
            print(row)
            
        row = '|---------|'
        for m in range(n_machines):
            row += f"------------|" \
                    f"------------|"
        print(row)

    
    return stop[n_jobs-1, n_machines-1]                                                       

In [35]:
PROC = generateData(machines=4, jobs=7, seed=2)

Compute the optimal job schedule:

In [37]:
seq, proctime = IP(PROC)
print(seq)
print(proctime)

[0, 2, 1, 6, 3, 5, 4]
52


Check the processing time and print the schedule

In [39]:
totalTime(PROC, seq)

52

In [40]:
totalTime(PROC, seq, log=True)

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    1 |            | Idle:    3 |            | Idle:    5 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   1 |            | Start:   3 |            | Start:   5 |
| Job:  0 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    6 |
|         |            | Stop:    1 |            | Stop:    3 |            | Stop:    5 |            | Stop:   11 |
|---------|------------|------------|------------|------------|---------

52

In [41]:
# Completing the Greedy Algorithm implementation
# Assuming the generateData function is already defined as in the notebook
def generateData(machines=4, jobs=7, seed=0):
    PROC = [ [ 0 for m in range(machines) ] for j in range(jobs) ]
    random.seed(seed)
    for j in range(jobs):
        for m in range(machines):
            PROC[j][m] = random.randint(1,9)
    return PROC

# Defining the Greedy Algorithm function
def greedy_job_scheduling(proc):
    num_jobs = len(proc)
    num_machines = len(proc[0])

    # Initial job order (you can start with any order)
    job_order = list(range(num_jobs))
    # Calculate the total time taken by each job across all machines
    total_times = [sum(proc[job]) for job in range(num_jobs)]
    
    # Sort jobs by their total processing time in ascending order
    job_order.sort(key=lambda x: total_times[x])
    
    return job_order

# Example usage
seed_value = 1234  # Example seed value; replace with your specific seed
proc = generateData(seed=seed_value)
schedule = greedy_job_scheduling(proc)
schedule


[2, 4, 1, 0, 6, 5, 3]

In [42]:
# Define the Genetic Algorithm
def genetic_algorithm_job_scheduling(proc, population_size=50, generations=100, mutation_rate=0.1, crossover_rate=0.9):
    num_jobs = len(proc)
    
    # Fitness function to evaluate a schedule
    def fitness(schedule):
        completion_times = [0] * len(proc[0])
        for job in schedule:
            for i in range(len(proc[job])):
                if i == 0:
                    completion_times[i] += proc[job][i]
                else:
                    completion_times[i] = max(completion_times[i], completion_times[i-1]) + proc[job][i]
        return -completion_times[-1]  # We want to minimize the total time, so use negative
    
    # Generate an initial population of random schedules
    def create_population():
        return [random.sample(range(num_jobs), num_jobs) for _ in range(population_size)]
    
    # Selection function using tournament selection
    def select_parents(population):
        return sorted(random.sample(population, 2), key=fitness)[0]
    
    # Crossover function to combine two parents
    def crossover(parent1, parent2):
        if random.random() > crossover_rate:
            return parent1[:]
        child = [-1] * num_jobs
        start, end = sorted(random.sample(range(num_jobs), 2))
        child[start:end] = parent1[start:end]
        pointer = 0
        for i in range(num_jobs):
            if parent2[i] not in child:
                while child[pointer] != -1:
                    pointer += 1
                child[pointer] = parent2[i]
        return child
    
    # Mutation function to introduce diversity
    def mutate(child):
        if random.random() < mutation_rate:
            i, j = random.sample(range(num_jobs), 2)
            child[i], child[j] = child[j], child[i]
    
    # Genetic Algorithm process
    population = create_population()
    for generation in range(generations):
        new_population = []
        for _ in range(population_size):
            parent1 = select_parents(population)
            parent2 = select_parents(population)
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)
        population = sorted(new_population, key=fitness)[:population_size]
    
    # Return the best solution found
    best_schedule = sorted(population, key=fitness)[0]
    return best_schedule

# Example usage
best_schedule_genetic = genetic_algorithm_job_scheduling(proc)
best_schedule_genetic


[2, 0, 5, 6, 4, 3, 1]

In [43]:
# Function to calculate the total processing time for a given job schedule
def calculate_total_time(proc, schedule):
    num_machines = len(proc[0])
    completion_times = [0] * num_machines
    for job in schedule:
        for i in range(num_machines):
            if i == 0:
                completion_times[i] += proc[job][i]
            else:
                completion_times[i] = max(completion_times[i], completion_times[i-1]) + proc[job][i]
    return completion_times[-1]

# Greedy Algorithm implementation (as previously defined)
def greedy_job_scheduling(proc):
    num_jobs = len(proc)
    num_machines = len(proc[0])

    # Initial job order (you can start with any order)
    job_order = list(range(num_jobs))
    # Calculate the total time taken by each job across all machines
    total_times = [sum(proc[job]) for job in range(num_jobs)]
    
    # Sort jobs by their total processing time in ascending order
    job_order.sort(key=lambda x: total_times[x])
    
    return job_order

# Genetic Algorithm implementation (as previously defined)
def genetic_algorithm_job_scheduling(proc, population_size=50, generations=100, mutation_rate=0.1, crossover_rate=0.9):
    num_jobs = len(proc)
    
    # Fitness function to evaluate a schedule
    def fitness(schedule):
        return -calculate_total_time(proc, schedule)  # We want to minimize the total time, so use negative
    
    # Generate an initial population of random schedules
    def create_population():
        return [random.sample(range(num_jobs), num_jobs) for _ in range(population_size)]
    
    # Selection function using tournament selection
    def select_parents(population):
        return sorted(random.sample(population, 2), key=fitness)[0]
    
    # Crossover function to combine two parents
    def crossover(parent1, parent2):
        if random.random() > crossover_rate:
            return parent1[:]
        child = [-1] * num_jobs
        start, end = sorted(random.sample(range(num_jobs), 2))
        child[start:end] = parent1[start:end]
        pointer = 0
        for i in range(num_jobs):
            if parent2[i] not in child:
                while child[pointer] != -1:
                    pointer += 1
                child[pointer] = parent2[i]
        return child
    
    # Mutation function to introduce diversity
    def mutate(child):
        if random.random() < mutation_rate:
            i, j = random.sample(range(num_jobs), 2)
            child[i], child[j] = child[j], child[i]
    
    # Genetic Algorithm process
    population = create_population()
    for generation in range(generations):
        new_population = []
        for _ in range(population_size):
            parent1 = select_parents(population)
            parent2 = select_parents(population)
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)
        population = sorted(new_population, key=fitness)[:population_size]
    
    # Return the best solution found
    best_schedule = sorted(population, key=fitness)[0]
    return best_schedule

# Generate processing time data
seed_value = 1234  # Example seed value; replace with your specific seed
proc = generateData(seed=seed_value)

# Run Greedy Algorithm
schedule_greedy = greedy_job_scheduling(proc)
total_time_greedy = calculate_total_time(proc, schedule_greedy)

# Run Genetic Algorithm
schedule_genetic = genetic_algorithm_job_scheduling(proc)
total_time_genetic = calculate_total_time(proc, schedule_genetic)

# Output the schedules and their processing times
print("Greedy Algorithm Schedule:", schedule_greedy)
print("Total Processing Time (Greedy):", total_time_greedy)

print("Genetic Algorithm Schedule:", schedule_genetic)
print("Total Processing Time (Genetic):", total_time_genetic)

# Determine which algorithm performed better
if total_time_greedy < total_time_genetic:
    print("Greedy Algorithm performed better.")
else:
    print("Genetic Algorithm performed better.")


Greedy Algorithm Schedule: [2, 4, 1, 0, 6, 5, 3]
Total Processing Time (Greedy): 60
Genetic Algorithm Schedule: [2, 0, 5, 6, 4, 3, 1]
Total Processing Time (Genetic): 66
Greedy Algorithm performed better.
